In [1]:
#| default_exp common._base_auto

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

# Hyperparameter Optimization

> Machine Learning forecasting methods are defined by many hyperparameters that control their behavior, with effects ranging from their speed and memory requirements to their predictive performance. For a long time, manual hyperparameter tuning prevailed. This approach is time-consuming, **automated hyperparameter optimization** methods have been introduced, proving more efficient than manual tuning, grid search, and random search.<br><br> The `BaseAuto` class offers shared API connections to hyperparameter optimization algorithms like [Optuna](https://docs.ray.io/en/latest/tune/examples/bayesopt_example.html), [HyperOpt](https://docs.ray.io/en/latest/tune/examples/hyperopt_example.html), [Dragonfly](https://docs.ray.io/en/latest/tune/examples/dragonfly_example.html) among others through `ray`, which gives you access to grid search, bayesian optimization and other state-of-the-art tools like hyperband.<br><br>Comprehending the impacts of hyperparameters is still a precious skill, as it can help guide the design of informed hyperparameter spaces that are faster to explore automatically.

![Figure 1. Example of dataset split (left), validation (yellow) and test (orange). The hyperparameter optimization guiding signal is obtained from the validation set.](imgs_models/data_splits.png)

In [3]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import show_doc

In [4]:
#| export
from copy import deepcopy
from os import cpu_count

import torch
import pytorch_lightning as pl

from pytorch_lightning.callbacks import TQDMProgressBar
from ray import air, tune
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.search.basic_variant import BasicVariantGenerator

from neuralforecast.losses.pytorch import MAE

/Users/cchallu/opt/anaconda3/envs/neuralforecast/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#| exporti
def train_tune(config_step, cls_model, dataset, val_size, test_size):
    metrics = {"loss": "ptl/val_loss"}
    callbacks = [TQDMProgressBar(), TuneReportCallback(metrics, on="validation_end")]
    if 'callbacks' in config_step.keys():
        callbacks += config_step['callbacks']
    config_step = {**config_step, **{'callbacks': callbacks}}
    model = cls_model(**config_step)
    model.fit(
        dataset,
        val_size=val_size, 
        test_size=test_size
    )

In [6]:
#| exporti
def tune_model(
        cls_model, 
        dataset, 
        val_size, 
        test_size,
        cpus,
        gpus,
        verbose,
        num_samples, 
        search_alg, 
        config
    ):
    train_fn_with_parameters = tune.with_parameters(
        train_tune,
        cls_model=cls_model,
        dataset=dataset,
        val_size=val_size,
        test_size=test_size,
    )

    # Device
    if gpus > 0:
        device_dict = {'gpu':gpus}
    else:
        device_dict = {'cpu':cpus}

    tuner = tune.Tuner(
        tune.with_resources(train_fn_with_parameters, device_dict),
        run_config=air.RunConfig(
            verbose=verbose,
            #checkpoint_config=air.CheckpointConfig(
                #num_to_keep=0,
                #keep_checkpoints_num=None
            #)
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            num_samples=num_samples, 
            search_alg=search_alg
        ),
        param_space=config,
    )
    results = tuner.fit()
    return results

In [7]:
#| export
class BaseAuto(pl.LightningModule):
    """ BaseAuto 
    
    Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to 
    give access to a wide variety of hyperparameter optimization tools ranging 
    from classic grid search, to Bayesian optimization and HyperBand algorithm.

    The validation loss to be optimized is defined by the `config['loss']` dictionary
    value, the config also contains the rest of the hyperparameter search space.

    It is important to note that the success of this hyperparameter optimization
    heavily relies on a strong correlation between the validation and test periods.

    **Parameters:**<br>
    `cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
    `h`: int, forecast horizon.<br>
    `config`: dict, dictionary with ray.tune defined search space.<br>
    `search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
        see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
    `num_samples`: int, number of hyperparameter optimization steps/samples.<br>
    `cpus`: int, number of cpus to use during optimization, default all available.<br>
    `gpus`: int, number of gpus to use during optimization, default all available.<br>
    `refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
    `verbose`: bool, wether print partial outputs.<br>
    """
    def __init__(self, 
                 cls_model,
                 h,
                 loss,
                 config, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 refit_with_val=False,
                 verbose=False):
        super(BaseAuto, self).__init__()
        self.save_hyperparameters() # Allows instantiation from a checkpoint from class

        if config.get('h', None) is not None:
            raise Exception("Please use `h` argument instead of `config['h']`.")
        if config.get('loss', None) is not None:
            raise Exception("Please use `loss` argument instead of `config['loss']`.")

        # Deepcopy to avoid modifying the original config
        config_base = deepcopy(config)

        config_base['h'] = h
        config_base['loss'] = loss
        self.cls_model = cls_model
        self.h = h
        self.loss = loss
        self.config = config_base
        self.num_samples = num_samples
        self.search_alg = search_alg
        self.cpus = cpus
        self.gpus = gpus
        self.refit_with_val = refit_with_val
        self.verbose = verbose
        self.loss = self.config.get('loss', MAE())
        
    def fit(self, dataset, val_size=0, test_size=0):
        """ BaseAuto.fit

        Perform the hyperparameter optimization as specified by the BaseAuto configuration 
        dictionary `config`.
        
        The optimization is performed on the `TimeSeriesDataset` using temporal cross validation with 
        the validation set that sequentially precedes the test set.

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset` see details [here](https://nixtla.github.io/neuralforecast/tsdataset.html)<br>
        `val_size`: int, size of temporal validation set (needs to be bigger than 0).<br>
        `test_size`: int, size of temporal test set (default 0).<br>

        **Returns:**<br>
        `self`: fitted instance of `BaseAuto` with best hyperparameters and results<br>.
        """
        #we need val_size > 0 to perform
        #hyperparameter selection.
        search_alg = deepcopy(self.search_alg)
        val_size = val_size if val_size > 0 else self.h
        results = tune_model(
            cls_model=self.cls_model,
            dataset=dataset,
            val_size=val_size, 
            test_size=test_size, 
            cpus=self.cpus,
            gpus=self.gpus,
            verbose=self.verbose,
            num_samples=self.num_samples, 
            search_alg=search_alg, 
            config=self.config
        )
        best_config = results.get_best_result().config
        self.model = self.cls_model(**best_config)
        self.model.fit(
            dataset=dataset, 
            val_size=val_size * (1 - self.refit_with_val), 
            test_size=test_size,
        )
        self.results = results
        
    def predict(self, dataset, step_size=1, **data_kwargs):
        """ BaseAuto.predict

        Predictions of the best performing model on validation.

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset` see details [here](https://nixtla.github.io/neuralforecast/tsdataset.html)<br>
        `step_size`: int, steps between sequential predictions, (default 1).<br>
        `**data_kwarg`: additional parameters for the dataset module.<br>

        **Returns:**<br>
        `y_hat`: numpy predictions of the `NeuralForecast` model.<br>
        """
        return self.model.predict(dataset=dataset, 
                                  step_size=step_size, **data_kwargs)

    def set_test_size(self, test_size):
        self.model.set_test_size(test_size)

    def save(self, path):
        """ BaseAuto.save

        Save the fitted model to disk.

        **Parameters:**<br>
        `path`: str, path to save the model.<br>
        """
        self.model.trainer.save_checkpoint(path)

In [8]:
show_doc(BaseAuto, title_level=3)

---

### BaseAuto

>      BaseAuto (cls_model, h, loss, config,
>                search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>                object at 0x121cefe20>, num_samples=10, cpus=12, gpus=0,
>                refit_with_val=False, verbose=False)

BaseAuto 

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to 
give access to a wide variety of hyperparameter optimization tools ranging 
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>

In [9]:
show_doc(BaseAuto.fit, title_level=3)

---

### BaseAuto.fit

>      BaseAuto.fit (dataset, val_size=0, test_size=0)

BaseAuto.fit

Perform the hyperparameter optimization as specified by the BaseAuto configuration 
dictionary `config`.

The optimization is performed on the `TimeSeriesDataset` using temporal cross validation with 
the validation set that sequentially precedes the test set.

**Parameters:**<br>
`dataset`: NeuralForecast's `TimeSeriesDataset` see details [here](https://nixtla.github.io/neuralforecast/tsdataset.html)<br>
`val_size`: int, size of temporal validation set (needs to be bigger than 0).<br>
`test_size`: int, size of temporal test set (default 0).<br>

**Returns:**<br>
`self`: fitted instance of `BaseAuto` with best hyperparameters and results<br>.

In [10]:
show_doc(BaseAuto.predict, title_level=3)

---

### BaseAuto.predict

>      BaseAuto.predict (dataset, step_size=1, **data_kwargs)

BaseAuto.predict

Predictions of the best performing model on validation.

**Parameters:**<br>
`dataset`: NeuralForecast's `TimeSeriesDataset` see details [here](https://nixtla.github.io/neuralforecast/tsdataset.html)<br>
`step_size`: int, steps between sequential predictions, (default 1).<br>
`**data_kwarg`: additional parameters for the dataset module.<br>

**Returns:**<br>
`y_hat`: numpy predictions of the `NeuralForecast` model.<br>

In [11]:
#| hide
import logging
import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [13]:
#| hide
import pandas as pd
from neuralforecast.models.mlp import MLP
from neuralforecast.utils import AirPassengersDF as Y_df
from neuralforecast.tsdataset import TimeSeriesDataset

Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test

dataset, *_ = TimeSeriesDataset.from_df(Y_train_df)
config = {
    "hidden_size": tune.choice([512]),
    "num_layers": tune.choice([3, 4]),
    "input_size": 12,
    "max_steps": 10,
    "val_check_steps": 1
}
auto = BaseAuto(h=12, loss=MAE(), cls_model=MLP, config=config, num_samples=2, cpus=1, gpus=0)
auto.fit(dataset=dataset)
y_hat = auto.predict(dataset=dataset)

(train_tune pid=20415) /Users/cchallu/opt/anaconda3/envs/neuralforecast/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(train_tune pid=20415)   warnings.warn('resource_tracker: There appear to be %d '
(train_tune pid=20416) /Users/cchallu/opt/anaconda3/envs/neuralforecast/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(train_tune pid=20416)   warnings.warn('resource_tracker: There appear to be %d '
(train_tune pid=20439) Global seed set to 1
(train_tune pid=20440) Global seed set to 1


Epoch 0:  50%|█████     | 1/2 [00:00<00:00, 23.05it/s, loss=250, v_num=0, train_loss_step=250.0]
Validation: 0it [00:00, ?it/s]0m 
Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 36.09it/s, loss=250, v_num=0, train_loss_step=250.0, val_loss=362.0]
                                                                       
Epoch 1:  50%|█████     | 1/2 [00:00<00:00, 25.56it/s, loss=232, v_num=0, train_loss_step=214.0, val_loss=362.0, train_loss_epoch=250.0]
Validation: 0it [00:00, ?it/s]0m 
Epoch 2:  50%|█████     | 1/2 [00:00<00:00, 25.55it/s, loss=211, v_num=0, train_loss_step=169.0, val_loss=284.0, train_loss_epoch=214.0]
Validation: 0it [00:00, ?it/s]0m 
Epoch 0:  50%|█████     | 1/2 [00:00<00:00, 20.62it/s, loss=250, v_num=0, train_loss_step=250.0]
Validation: 0it [00:00, ?it/s]0m 
Epoch 1:  50%|█████     | 1/2 [00:00<00:00, 24.92it/s, loss=232, v_num=0, train_loss_step=214.0, val_loss=362.0, train_loss_epoch=250.0]
Validation: 0it [00:00, ?it/s]0m 
Epoch 3:  50%|█████     | 1/2 [00:00<00:

(train_tune pid=20440) /Users/cchallu/opt/anaconda3/envs/neuralforecast/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(train_tune pid=20440)   warnings.warn('resource_tracker: There appear to be %d '
(train_tune pid=20439) /Users/cchallu/opt/anaconda3/envs/neuralforecast/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(train_tune pid=20439)   warnings.warn('resource_tracker: There appear to be %d '
Global seed set to 1


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 87.50it/s] 


In [ ]:
#| hide
Y_test_df['AutoMLP'] = y_hat

pd.concat([Y_train_df, Y_test_df]).drop('unique_id', axis=1).set_index('ds').plot()

### References
- [James Bergstra, Remi Bardenet, Yoshua Bengio, and Balazs Kegl (2011). "Algorithms for Hyper-Parameter Optimization". In: Advances in Neural Information Processing Systems. url: https://proceedings.neurips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf](https://proceedings.neurips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf)
- [Kirthevasan Kandasamy, Karun Raju Vysyaraju, Willie Neiswanger, Biswajit Paria, Christopher R. Collins, Jeff Schneider, Barnabas Poczos, Eric P. Xing (2019). "Tuning Hyperparameters without Grad Students: Scalable and Robust Bayesian Optimisation with Dragonfly". Journal of Machine Learning Research. url: https://arxiv.org/abs/1903.06694](https://arxiv.org/abs/1903.06694)
- [Lisha Li, Kevin Jamieson, Giulia DeSalvo, Afshin Rostamizadeh, Ameet Talwalkar (2016). "Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization". Journal of Machine Learning Research. url: https://arxiv.org/abs/1603.06560](https://arxiv.org/abs/1603.06560)